# Get Data

In [9]:
import numpy as np
import pandas as pd

from sklearn import datasets

iris = datasets.load_iris()

%pprint off

Pretty printing has been turned OFF


In [10]:
list(iris.keys())

['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module']

In [11]:
attributes = iris["feature_names"]
attributes

['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']

So we have 4 features all in centimeters.

In [14]:
classes = iris["target_names"]
classes

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

These are the classes in order 0, 1, 2

In [4]:
X = iris["data"]

y = iris["target"]

In [5]:
X.shape, y.shape

((150, 4), (150,))

## Add X<sub>0</sub> to X

In [23]:
X = np.c_[np.ones((X.shape[0], 1)), X]

Let's also define our number of classes.

In [85]:
classes = np.unique(y).shape[0]
classes

3

In [24]:
X[:3, :]

array([[1. , 5.1, 3.5, 1.4, 0.2],
       [1. , 4.9, 3. , 1.4, 0.2],
       [1. , 4.7, 3.2, 1.3, 0.2]])

In [19]:
np.unique(y, return_counts=True)

(array([0, 1, 2]), array([50, 50, 50]))

No need to set aside a test set, as we need to first implement a working softmax model and then run a learning curve function on the model.

# Softmax Functions

We need to do the following:
- Random initialization of n+1 weights and k times where k=3 the number of classes (3 flower types)
    - Each weight should be a vector that has 5 values and we should have 3 vectors
- Compute softmax score for each class k
- Compute probability score for each softmax score
- Compute cross entropy gradient vector for each class k (should be same vector as weights - n+1 by 1 vector)
- Combine all of the above into a Batch Gradient Descent Step function
    - Args: initial 3 weight vectors, training data (m by n), label data (m by 1), learning rate
    - Append a vector of 1s to training data (m by n+1)
    - calculate softmax scores for each class
    - calculate probability scores for each class
    - calculate gradient vector for each class
    - subtract learning rate x gradient vector from initial weights
    - return the 3 new weights

## Weight Initialization

In [198]:
def random_init(classes):
    init_thetas = []
    for i in range(0,classes):
        init_thetas.append(np.random.randn(5))
    return np.array(init_thetas)
    
init_thetas = random_init(classes)
init_thetas

array([[-0.47644867, -1.15820463,  0.99975192,  0.46776206, -0.48977563],
       [ 1.1661659 ,  1.20399017, -1.11875637,  1.36848649,  0.58877015],
       [ 1.12410547, -1.82449253, -0.46877284,  0.08343482,  1.49358173]])

In [111]:
init_thetas.shape

(3, 5)

We have a list of random 5 by 1 vector weights and one for each of the three flower classes.

## Softmax Scores

We need a function that, given one instance of X: x, calculates the softmax score of each class using the initial weights.
- calculate softmax score for one instance
- do it for each array in thetas

In [44]:
example_x = X[0]
example_x

array([1. , 5.1, 3.5, 1.4, 0.2])

Let's use this instance to test our math functions.

In [115]:
def softmax_k(x, theta_k):
    sm_score = theta_k.T.dot(x)
    return sm_score

In [116]:
example_sm_k = softmax_k(example_x, init_thetas[0])
example_sm_k

-3.7671119089056218

Note that it is automatically a numpy array of one, instead of just a float.

In [117]:
def softmax_scores(x, thetas):
    sm_scores = []
    for theta in thetas:
        sm_scores.append(softmax_k(x, theta))
    return np.array(sm_scores)

In [118]:
example_sm_scores = softmax_scores(example_x, init_thetas)
example_sm_scores

array([ -3.76711191,   1.43464903, -10.94172301])

## Probability Scores

In [119]:
def prob_k(sm_k, sm_scores):
    return np.exp(sm_k) / np.sum(np.exp(sm_scores))

In [120]:
example_prob_k = prob_k(example_sm_k, example_sm_scores)
example_prob_k

0.005476676260672716

In [121]:
def prob_scores(sm_scores):
    prob_scores = []
    for sm_k in sm_scores:
        prob_scores.append(prob_k(sm_k, sm_scores))
    return np.array(prob_scores)

In [122]:
example_prob_scores = prob_scores(example_sm_scores)
example_prob_scores

array([5.47667626e-03, 9.94519130e-01, 4.19394816e-06])

This should be accurate.

In [123]:
np.sum(example_prob_scores)

1.0

This makes sense, since the total probability should always be 1

## Gradient Vectors

In [134]:
def batch_grad(X, y, thetas):
    # compute all softmax scores for all instances for all classes 
    # m by k (150 by 3)
    sm = []
    for x in X:
        sm.append(softmax_scores(x, thetas))
    sm = np.array(sm)
    # indeed a (150, 3) numpy array
        
    # compute all prob scores for all instances for all classes
    # should also be (150 by 3)
    pb = []
    for scores in sm:
        pb.append(prob_scores(scores))
    pb = np.array(pb)
    # indeed (150, 3) and sum across columns produces a (150,) array of 1s
    
    # compute gradient vector for first class or y=0
    grad_vec = []
    for k in range(thetas.shape[0]):
        errs = []
        for i in range(X.shape[0]):
            err = (pb[i, k] - (y[i] == k))
            # err * X[i] is a (1, 5) or 1 by n+1 row vector
            errs.append(err * X[i])
        # errs is a (150, 5) matrix of errors for each feature value for each instance
        grad_k = np.mean(errs, axis=0)
        grad_vec.append(grad_k)
    return np.array(grad_vec)

In [135]:
batch_grad(X, y, init_thetas)

array([[-0.33118776, -1.65813918, -1.13551034, -0.4840576 , -0.08140887],
       [ 0.66451877,  3.85412818,  2.12683645,  2.33472103,  0.75674166],
       [-0.33333101, -2.195989  , -0.99132611, -1.85066343, -0.67533279]])

So we got our gradient vector. Now we just need to iterate.

## Prediction Function

In [136]:
def predict_single(x, thetas):
    sm = softmax_scores(x, thetas)
    pb = prob_scores(sm)
    return np.argmax(pb)

In [137]:
predict_single(X[0], init_thetas)

1

In [138]:
y[0]

0

So we got it wrong, but this is expected - let's write predict for arrays.

In [176]:
def predict_aux(X, thetas):
    predictions = []
    for x in X:
        predictions.append(predict_single(x, thetas))
    return np.array(predictions)

In [177]:
predict_aux(X, init_thetas)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

This is obviously wrong, but let's see if it will be more accurate after we train our model.

# Training Model

## Step Function

We need our function to iteratively improve and minimize cost.
- Calculate gradient vectors
- Subtract by a constant learning rate x gradient vector for each classes' theta
- Do this epoch times and then return final thetas

In [154]:
init_thetas

array([[ 1.48310779, -0.74911548, -0.09355695, -0.74296951, -0.31062059],
       [ 0.73858013,  0.58813831, -0.62268093, -0.06733114, -0.14894813],
       [ 0.60105558, -0.40502176, -1.89907845, -2.03860217,  0.1182498 ]])

In [155]:
def batch_step(X, y, init_thetas, l_rate, epochs):
    thetas = init_thetas.copy()
    for e in range(epochs):
        # thetas is a list of k arrays and each array is a 5 by 1 array
        grad_vecs = batch_grad(X, y, thetas)
        # (3, 5) - l_rate * (3, 5)
        thetas = thetas - (l_rate * grad_vecs)
    return thetas

In [159]:
thetas_10e = batch_step(X, y, init_thetas, 0.1, 10)
thetas_10e

array([[ 1.51961083, -0.69133747,  0.08670604, -1.05299478, -0.45339878],
       [ 0.59884774, -0.27782354, -1.1129471 , -0.63705921, -0.36053198],
       [ 0.70428494,  0.40316208, -1.58907526, -1.15884882,  0.47261183]])

In [165]:
thetas_1000e = batch_step(X, y, init_thetas, 0.1, 1000)
thetas_1000e

array([[ 1.87843081,  0.1345169 ,  1.7355029 , -3.42539106, -1.56533779],
       [ 1.29785131,  0.68355838, -1.19022508, -1.08241329, -1.45042904],
       [-0.35353861, -1.3840742 , -3.16059414,  1.65890153,  2.67444791]])

## Test New Thetas

In [178]:
predict_aux(X, thetas_10e)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0,
       0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [179]:
predictions_1000e = predict_aux(X, thetas_1000e)
predictions_1000e

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [180]:
from sklearn.metrics import accuracy_score

accuracy_score(predictions_1000e, y)

0.9733333333333334

Wow that's pretty good!

# Make Softmax Class

Our next step is to include the training process in a class that we can fit and transform using sklearn.
- create class framework
- private variable and argument max_iter_ that records number of iterations per transform 
- private variable thetas_ that tracks the newest weights
- use batch_step in transform

In [231]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import unique_labels

class SoftmaxClassifier(BaseEstimator, ClassifierMixin):
    
    def __init__(self, max_iter=10, l_rate=0.05):
        self.max_iter = max_iter
        self.l_rate = l_rate
        
        # init theta
        self.thetas_ = None
        
    def fit(self, X, y):
        # Check that X and y have correct shape
        X, y = check_X_y(X, y)
        
        # Store the classes seen during fit
        self.classes_ = unique_labels(y)
        
        self.X_ = X
        self.y_ = y
        
        # initialize weights
        if self.thetas_ is None:
            self.thetas_ = random_init(len(self.classes_))
        
        # train weights and update theta
        self.thetas_ = batch_step(self.X_, self.y_, self.thetas_, self.l_rate, self.max_iter)
        
        return self
        
    def predict(self, X):
        # Check if fit has been called
        check_is_fitted(self)
        
        # Input validation
        X = check_array(X)
        
        return predict_aux(X, self.thetas_)

In [232]:
sm_clf = SoftmaxClassifier(max_iter=1000, l_rate=0.1)

In [233]:
sm_clf.fit(X, y)

SoftmaxClassifier(l_rate=0.1, max_iter=1000)

In [234]:
predictions = sm_clf.predict(X)
predictions

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [235]:
accuracy_score(predictions, y)

0.98

# Implement Early Stopping

Now we need to implement early stopping.
- Split X, y into training and validation sets
- Loop for a number of epochs and in each loop:
    - fit model on training set
    - predict on validation set
    - get MSE of predictions
    - compare MSE to current min MSE
        - if less then update MSE, best_epoch, and best_model (with deepcopy)
        - else keep a counter of how many times it has not beat current min MSE
            - if counter exceeds a certain count (stop_count) then exit loop

In [247]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from copy import deepcopy

# 1 iteration epoch
sm_clf = SoftmaxClassifier(max_iter=1, l_rate=0.1)

# after how many non-improving iterations to stop
max_stop_count = 1000
current_stop_count = 0

min_MSE = float("inf")
best_epoch = None
best_model = None

# split into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

for epoch in range(10000):
    if current_stop_count >= max_stop_count:
        break
    sm_clf.fit(X_train, y_train)
    predictions = sm_clf.predict(X_val)
    MSE = mean_squared_error(predictions, y_val)
    if MSE < min_MSE:
        min_MSE = MSE
        best_epoch = epoch
        best_model = deepcopy(sm_clf)
    else:
        current_stop_count += 1

In [248]:
best_epoch

177

In [249]:
final_predictions = best_model.predict(X)
accuracy_score(final_predictions, y)

0.9733333333333334

# Conclusion

So Softmax is a multiclass regression method that uses gradient descent.

It calculate softmax scores for each instance for each class. 
Then it calculates the probability scores using those softmax scores.
Finally, it uses the probability scores and the label to calculate error.

Using this method and cross-entropy - a gradient vector for each class is calculated.
Subtract this gradient vector from the previous weights to get new weights.

Iterate to improve the model.

Early stopping is also quite easy - iterate until model stops improving for some amount of time.

I believe early stopping would have better results if our dataset was in the thousands or tens of thousands, sadly the iris dataset is only 150 instances.